In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from datetime import datetime
from functools import reduce
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
hsr_lower_zone = pd.read_csv('upper_half_full.csv')
hsr_lower_zone_hexes = hsr_lower_zone['pickup_location_hex_8'].unique().tolist()
print(hsr_lower_zone_hexes)
print(len(hsr_lower_zone_hexes))

['886189258dfffff', '8861892513fffff', '88618925c3fffff', '88618925ebfffff', '88618925cbfffff', '88618925d1fffff', '886189258bfffff', '8861892425fffff', '88618925e3fffff', '8861892589fffff', '886189251bfffff', '8861892599fffff', '88618925c7fffff', '8861892427fffff', '8861892587fffff', '8861892585fffff', '88618925ddfffff', '8861892581fffff', '88618925d3fffff', '88618925c9fffff', '88618925c5fffff', '88618925d5fffff', '886189259dfffff', '88618925e9fffff', '88618925e1fffff', '88618925cdfffff', '8861892583fffff', '88618925c1fffff', '88618925d7fffff', '8861892519fffff', '8861892517fffff', '88618925edfffff', '8861892511fffff']
33


In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8
from orders.order_logs_snapshot
where yyyymmdd >= '20230410'
and yyyymmdd <= '20230423'
and city_name in ('Bangalore')
and service_obj_service_name = 'Link'
and pickup_location_hex_8 in ('8861892545fffff', '886189254bfffff', '8861892541fffff', '88618920d5fffff', '8861892099fffff', '8861892085fffff', '886189255dfffff',
 '8861892087fffff', '8861892543fffff', '88618920e3fffff', '88618920bbfffff', '886189254dfffff', '88618920dbfffff', '8861892729fffff', '8861892089fffff',
  '8861892093fffff', '886189209bfffff', '88618920abfffff', '8861892767fffff', '8861892081fffff', '88618920a9fffff', '886189208dfffff', '886189272dfffff',
   '8861892723fffff', '8861892083fffff', '8861892549fffff', '88618920b9fffff', '8861892091fffff', '8861892095fffff', '8861892721fffff', '88618920b1fffff',
    '8861892727fffff', '886189209dfffff', '8861892097fffff', '8861892725fffff', '88618920bdfffff')
"""


#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [4]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8
0,643bb6b1ee623a70eee720c4,1.507,Bangalore,8861892085fffff,12.947007,77.722290,141953,14,1415,Link,14:00,7,20230416,dropped,None,6.0,"[""63ee2f84e370d6dd6c59ef90"",""6162e9ef77c7e6b1e...",8861892085fffff
1,643bb89b6b8317383f414cab,1.853,Bangalore,88618920bbfffff,12.950012,77.701265,142803,14,1415,Link,14:00,7,20230416,dropped,None,8.0,"[""623dec2579d2674e4fb07f51"",""607a6153255e27cc6...",88618920bbfffff
2,643bbbf2e0382b532b65ba26,1.789,Bangalore,88618920b1fffff,12.956978,77.702780,144218,14,1430,Link,14:59,7,20230416,customerCancelled,Customer asked to cancel,7.0,"[""5e4b95927ea425a39b6cb910"",""63c8b353ec19f00c6...",88618920b1fffff
3,643bbcf8cac0523bb40a0e9f,1.583,Bangalore,886189254bfffff,12.919333,77.671005,144640,14,1445,Link,14:59,7,20230416,dropped,None,6.0,"[""63d1e080e00e30fe287716ef"",""606f124d7761609b7...",886189254bfffff
4,643bbdb8259b2f4d1a7b4cb9,NaN,Bangalore,88618920b1fffff,12.954740,77.700409,144952,14,1445,Link,14:59,7,20230416,customerCancelled,order cancelled before rider accepted,0.0,"[""63ff5aac24d5dc39dfcddaa9"",""5ca7164a54bc7263f...",88618920b1fffff


In [5]:
#View the dataset
df_captain = df.copy()
df_captain.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8
0,643bb6b1ee623a70eee720c4,1.507,Bangalore,8861892085fffff,12.947007,77.722290,141953,14,1415,Link,14:00,7,20230416,dropped,None,6.0,"[""63ee2f84e370d6dd6c59ef90"",""6162e9ef77c7e6b1e...",8861892085fffff
1,643bb89b6b8317383f414cab,1.853,Bangalore,88618920bbfffff,12.950012,77.701265,142803,14,1415,Link,14:00,7,20230416,dropped,None,8.0,"[""623dec2579d2674e4fb07f51"",""607a6153255e27cc6...",88618920bbfffff
2,643bbbf2e0382b532b65ba26,1.789,Bangalore,88618920b1fffff,12.956978,77.702780,144218,14,1430,Link,14:59,7,20230416,customerCancelled,Customer asked to cancel,7.0,"[""5e4b95927ea425a39b6cb910"",""63c8b353ec19f00c6...",88618920b1fffff
3,643bbcf8cac0523bb40a0e9f,1.583,Bangalore,886189254bfffff,12.919333,77.671005,144640,14,1445,Link,14:59,7,20230416,dropped,None,6.0,"[""63d1e080e00e30fe287716ef"",""606f124d7761609b7...",886189254bfffff
4,643bbdb8259b2f4d1a7b4cb9,NaN,Bangalore,88618920b1fffff,12.954740,77.700409,144952,14,1445,Link,14:59,7,20230416,customerCancelled,order cancelled before rider accepted,0.0,"[""63ff5aac24d5dc39dfcddaa9"",""5ca7164a54bc7263f...",88618920b1fffff


In [6]:
#Filter only on the hexes needed
df_filter_level1 = df
df_filter_level1['count'] = 1
df_filter_level1['date'] = df_filter_level1['yyyymmdd'].astype(str).apply(lambda x: x[6:])

In [7]:
#Create a new column for cancel_reason to identify OCARA
df_filter_level1['new_cancel_reason'] = np.where(df_filter_level1['order_status'] == 'customerCancelled',
 np.where(df_filter_level1['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_filter_level1['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')

In [8]:
#Check if the order is expiry_mapped or expiry_unmapped
df_filter_level1['expiry_type'] = np.where(df_filter_level1['order_status'] == 'expired',
 np.where(df_filter_level1['map_riders'] == '[]', 'expiry_unmapped', 'expiry_mapped'), 'no_expiry')
df_filter_level1['final_order_status'] = np.where(df_filter_level1['order_status'] == 'customerCancelled', df_filter_level1['new_cancel_reason'],
np.where(df_filter_level1['order_status'] == 'expired', df_filter_level1['expiry_type'], df_filter_level1['order_status']))

In [9]:
#Metrics_pivot
metrics_pivot = pd.pivot_table(df_filter_level1, index = ['yyyymmdd', 'hour'], columns = 'final_order_status',
 values = 'count', aggfunc = 'sum', fill_value=0).reset_index().rename_axis(None, axis=1)
metrics_pivot['total'] = metrics_pivot[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expiry_mapped', 'expiry_unmapped']].sum(axis = 1)
metrics_pivot['OCARA%'] = round(metrics_pivot['OCARA']/metrics_pivot['total'], 4)
metrics_pivot['COBRA%'] = round(metrics_pivot['COBRA']/metrics_pivot['total'], 4)
metrics_pivot['COBRM%'] = round(metrics_pivot['COBRM']/metrics_pivot['total'], 4)
metrics_pivot['G2N'] = round(metrics_pivot['dropped']/metrics_pivot['total'], 4)
metrics_pivot['expiry_mapped%'] = round(metrics_pivot['expiry_mapped']/metrics_pivot['total'], 4)
if 'expiry_unmapped' in metrics_pivot.columns:
    metrics_pivot['Stockout%'] = round(metrics_pivot['expiry_unmapped']/metrics_pivot['total'], 4)
else:
    metrics_pivot['Stockout%'] = 0
metrics_df = metrics_pivot[['yyyymmdd', 'hour', 'OCARA%', 'COBRA%', 'COBRM%', 'G2N', 'expiry_mapped%', 'Stockout%', 'total']]
metrics_df.head()

,yyyymmdd,hour,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total
0,20230410,00,0.3367,0.2551,0.0000,0.3929,0.0102,0.0,196
1,20230410,01,0.2917,0.0833,0.0000,0.5833,0.0417,0.0,48
2,20230410,02,0.4167,0.1111,0.0278,0.4167,0.0278,0.0,36
3,20230410,03,0.2903,0.2581,0.1613,0.2419,0.0323,0.0,62
4,20230410,04,0.2000,0.2889,0.0000,0.4222,0.0889,0.0,45


In [10]:
#Get the average ETA and FM
df_dropped = df_filter_level1[df_filter_level1['final_order_status'] == 'dropped']
fm = df_dropped.groupby(['yyyymmdd', 'hour'])['accept_to_pickup_distance'].mean().reset_index()
fm = fm.rename(columns = {'accept_to_pickup_distance':'FM'})
eta = df_dropped.groupby(['yyyymmdd', 'hour'])['eta'].mean().reset_index()
metrics_df = metrics_df.merge(fm, on = ['yyyymmdd', 'hour'], how = 'left')
metrics_df = metrics_df.merge(eta, on = ['yyyymmdd', 'hour'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta
0,20230410,00,0.3367,0.2551,0.0000,0.3929,0.0102,0.0,196,1.260364,4.857143
1,20230410,01,0.2917,0.0833,0.0000,0.5833,0.0417,0.0,48,1.633357,5.857143
2,20230410,02,0.4167,0.1111,0.0278,0.4167,0.0278,0.0,36,1.453200,6.133333
3,20230410,03,0.2903,0.2581,0.1613,0.2419,0.0323,0.0,62,1.426933,5.066667
4,20230410,04,0.2000,0.2889,0.0000,0.4222,0.0889,0.0,45,2.018526,6.105263


In [11]:
#Get the number of captains by cluster
#Make columns for second and minute and date
df_captain['second'] = df_captain['hhmmss'].astype(str).apply(lambda x: x[4:])
df_captain['minute'] = df_captain['hhmmss'].astype(str).apply(lambda x: x[2:4])
df_captain['date'] = df_captain['yyyymmdd'].astype(str).apply(lambda x: x[6:])
df_captain['map_riders'] = df_captain['map_riders'].apply(lambda x: json.loads(x))
df_captain = df_captain[df_captain['map_riders'].apply(lambda x: len(x)) > 0]
df_captain['15_sec_batch'] = pd.cut(df_captain['second'].astype(int), bins = [-1, 15, 30, 45, 61], labels = ['0-15', '15-30', '30-45', '45-60'])

#Get the riders mapped in 20_sec_windows
df_captains_20_sec = df_captain.groupby(['yyyymmdd', 'hour', 'minute', '15_sec_batch'])['map_riders'].apply(list).reset_index()

#Get the number of captains in a single list
map_riders = []
df_captains_20_sec = df_captains_20_sec.dropna()
for x in df_captains_20_sec['map_riders'].values:
    riders = [item for sublist in x for item in sublist]
    map_riders.append(riders)

unique_captains = []
for x in map_riders:
    unique_captains_list = []
    for y in x:
        if y in unique_captains_list:
            continue
        else:
            unique_captains_list.append(y)
    unique_captains.append(unique_captains_list)

df_captains_20_sec['total_captains'] = unique_captains

#Get the number of captains in each batch
df_captains_20_sec['number_of_captains'] = df_captains_20_sec['total_captains'].apply(lambda x: len(x))

#Get the average number of captains coming in each 20_second_window
df_20_sec_average = df_captains_20_sec.groupby(['yyyymmdd', 'hour'])['number_of_captains'].median().reset_index()

#Merge the hourly captains and 20_sec_captains with the metrics table
metrics_df = metrics_df.merge(df_20_sec_average, on = ['yyyymmdd', 'hour'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains
0,20230410,00,0.3367,0.2551,0.0000,0.3929,0.0102,0.0,196,1.260364,4.857143,25.0
1,20230410,01,0.2917,0.0833,0.0000,0.5833,0.0417,0.0,48,1.633357,5.857143,23.0
2,20230410,02,0.4167,0.1111,0.0278,0.4167,0.0278,0.0,36,1.453200,6.133333,13.0
3,20230410,03,0.2903,0.2581,0.1613,0.2419,0.0323,0.0,62,1.426933,5.066667,8.0
4,20230410,04,0.2000,0.2889,0.0000,0.4222,0.0889,0.0,45,2.018526,6.105263,5.5


In [12]:
#Query data from order_logs_immutable for calculating TTC

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, pickup_location_hex_8
from orders.order_logs_immutable
where yyyymmdd >= '20230410'
and yyyymmdd <= '20230423'
and city_name in ('Bangalore')
and service_obj_service_name = 'Link'
and event_type in ('customer_cancelled', 'order_requested', 'accepted')
and pickup_location_hex_8 in ('8861892545fffff', '886189254bfffff', '8861892541fffff', '88618920d5fffff', '8861892099fffff', '8861892085fffff', '886189255dfffff',
 '8861892087fffff', '8861892543fffff', '88618920e3fffff', '88618920bbfffff', '886189254dfffff', '88618920dbfffff', '8861892729fffff', '8861892089fffff',
  '8861892093fffff', '886189209bfffff', '88618920abfffff', '8861892767fffff', '8861892081fffff', '88618920a9fffff', '886189208dfffff', '886189272dfffff',
   '8861892723fffff', '8861892083fffff', '8861892549fffff', '88618920b9fffff', '8861892091fffff', '8861892095fffff', '8861892721fffff', '88618920b1fffff',
    '8861892727fffff', '886189209dfffff', '8861892097fffff', '8861892725fffff', '88618920bdfffff')
"""

#Load data into pandas table
df_ttc_copy = pd.read_sql(q, connection)

In [13]:
#Load and view dataset
df_ttc = df_ttc_copy.copy()
df_ttc['timestamp'] = df_ttc['updated_epoch'].apply(lambda x: datetime.fromtimestamp(x/1000))
df_ttc = df_ttc.sort_values(by = 'timestamp')
df_ttc.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,pickup_location_hex_8,timestamp
382996,643304506b042d72ccd30197,NaN,,NaN,NaN,Bangalore,88618920b1fffff,8861892cc7fffff,13.037849,77.617166,12.952156,77.701286,000040,00,0000,Link,00:00,1,20230410,requested,None,1681065040845,order_requested,88618920b1fffff,2023-04-10 00:00:40.845
420498,643304506b042d72ccd30197,NaN,88618920b1fffff,12.955117,77.700855,Bangalore,88618920b1fffff,8861892cc7fffff,13.037849,77.617166,12.952156,77.701286,000040,00,0000,Link,00:00,1,20230410,onTheWay,None,1681065049505,accepted,88618920b1fffff,2023-04-10 00:00:49.505
392950,64330485ddfc230482e14b7d,NaN,,NaN,NaN,Bangalore,8861892097fffff,886189209dfffff,12.936372,77.705558,12.942347,77.697276,000133,00,0000,Link,00:00,1,20230410,requested,None,1681065093894,order_requested,8861892097fffff,2023-04-10 00:01:33.894
405472,643304946b042d72ccd301de,NaN,,NaN,NaN,Bangalore,8861892723fffff,8861892cb3fffff,13.058752,77.581215,12.908463,77.684993,000148,00,0000,Link,00:00,1,20230410,requested,None,1681065108607,order_requested,8861892723fffff,2023-04-10 00:01:48.607
397125,64330495373d85214287ba30,NaN,,NaN,NaN,Bangalore,88618920b9fffff,8861892097fffff,12.940368,77.696079,12.945477,77.706825,000149,00,0000,Link,00:00,1,20230410,requested,None,1681065109254,order_requested,88618920b9fffff,2023-04-10 00:01:49.254


In [14]:
#Create a column to identify OCARA, COBRA and COBRM
df_ttc['new_cancel_reason'] = np.where(df_ttc['event_type'] == 'customer_cancelled',
 np.where(df_ttc['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_ttc['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')
df_ttc.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,pickup_location_hex_8,timestamp,new_cancel_reason
382996,643304506b042d72ccd30197,NaN,,NaN,NaN,Bangalore,88618920b1fffff,8861892cc7fffff,13.037849,77.617166,12.952156,77.701286,000040,00,0000,Link,00:00,1,20230410,requested,None,1681065040845,order_requested,88618920b1fffff,2023-04-10 00:00:40.845,no_cancellation
420498,643304506b042d72ccd30197,NaN,88618920b1fffff,12.955117,77.700855,Bangalore,88618920b1fffff,8861892cc7fffff,13.037849,77.617166,12.952156,77.701286,000040,00,0000,Link,00:00,1,20230410,onTheWay,None,1681065049505,accepted,88618920b1fffff,2023-04-10 00:00:49.505,no_cancellation
392950,64330485ddfc230482e14b7d,NaN,,NaN,NaN,Bangalore,8861892097fffff,886189209dfffff,12.936372,77.705558,12.942347,77.697276,000133,00,0000,Link,00:00,1,20230410,requested,None,1681065093894,order_requested,8861892097fffff,2023-04-10 00:01:33.894,no_cancellation
405472,643304946b042d72ccd301de,NaN,,NaN,NaN,Bangalore,8861892723fffff,8861892cb3fffff,13.058752,77.581215,12.908463,77.684993,000148,00,0000,Link,00:00,1,20230410,requested,None,1681065108607,order_requested,8861892723fffff,2023-04-10 00:01:48.607,no_cancellation
397125,64330495373d85214287ba30,NaN,,NaN,NaN,Bangalore,88618920b9fffff,8861892097fffff,12.940368,77.696079,12.945477,77.706825,000149,00,0000,Link,00:00,1,20230410,requested,None,1681065109254,order_requested,88618920b9fffff,2023-04-10 00:01:49.254,no_cancellation


In [15]:
#Filter the df only zone1
df_ttc_filter = df_ttc
df_ttc_filter.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,pickup_location_hex_8,timestamp,new_cancel_reason
382996,643304506b042d72ccd30197,NaN,,NaN,NaN,Bangalore,88618920b1fffff,8861892cc7fffff,13.037849,77.617166,12.952156,77.701286,000040,00,0000,Link,00:00,1,20230410,requested,None,1681065040845,order_requested,88618920b1fffff,2023-04-10 00:00:40.845,no_cancellation
420498,643304506b042d72ccd30197,NaN,88618920b1fffff,12.955117,77.700855,Bangalore,88618920b1fffff,8861892cc7fffff,13.037849,77.617166,12.952156,77.701286,000040,00,0000,Link,00:00,1,20230410,onTheWay,None,1681065049505,accepted,88618920b1fffff,2023-04-10 00:00:49.505,no_cancellation
392950,64330485ddfc230482e14b7d,NaN,,NaN,NaN,Bangalore,8861892097fffff,886189209dfffff,12.936372,77.705558,12.942347,77.697276,000133,00,0000,Link,00:00,1,20230410,requested,None,1681065093894,order_requested,8861892097fffff,2023-04-10 00:01:33.894,no_cancellation
405472,643304946b042d72ccd301de,NaN,,NaN,NaN,Bangalore,8861892723fffff,8861892cb3fffff,13.058752,77.581215,12.908463,77.684993,000148,00,0000,Link,00:00,1,20230410,requested,None,1681065108607,order_requested,8861892723fffff,2023-04-10 00:01:48.607,no_cancellation
397125,64330495373d85214287ba30,NaN,,NaN,NaN,Bangalore,88618920b9fffff,8861892097fffff,12.940368,77.696079,12.945477,77.706825,000149,00,0000,Link,00:00,1,20230410,requested,None,1681065109254,order_requested,88618920b9fffff,2023-04-10 00:01:49.254,no_cancellation


In [16]:
#Filter df on the columns needed
df_filter = df_ttc_filter[['order_id', 'pickup_location_hex_8', 'hour', 'yyyymmdd', 'timestamp', 'event_type', 'new_cancel_reason']]

#Filter on the respective order_status
df_cobra = df_filter[df_filter['new_cancel_reason'] == 'COBRA']
df_ocara = df_filter[df_filter['new_cancel_reason'] == 'OCARA']
df_cobrm = df_filter[df_filter['new_cancel_reason'] == 'COBRM']
df_order_created = df_filter[df_filter['event_type'] == 'order_requested']
df_accepted = df_filter[df_filter['event_type'] == 'accepted']

#Sort the dfs by timestamp
df_cobra = df_cobra.sort_values(by = 'timestamp', ascending=True)
df_cobra = df_cobra.drop_duplicates(subset='order_id')

df_ocara = df_ocara.sort_values(by = 'timestamp', ascending=True)
df_ocara = df_ocara.drop_duplicates(subset='order_id')

df_cobrm = df_cobrm.sort_values(by = 'timestamp', ascending=True)
df_cobrm = df_cobrm.drop_duplicates(subset='order_id')

df_accepted = df_accepted.sort_values(by = 'timestamp', ascending=True)
df_accepted = df_accepted.drop_duplicates(subset='order_id')

#Change the column names
df_cobra = df_cobra[['order_id', 'new_cancel_reason', 'timestamp']]
df_cobra.columns = ['order_id', 'COBRA', 'timestamp_cobra']

df_cobrm = df_cobrm[['order_id', 'new_cancel_reason', 'timestamp']]
df_cobrm.columns = ['order_id', 'COBRM', 'timestamp_cobrm']

df_ocara = df_ocara[['order_id', 'new_cancel_reason', 'timestamp']]
df_ocara.columns = ['order_id', 'OCARA', 'timestamp_ocara']

df_accepted = df_accepted[['order_id', 'event_type', 'timestamp']]
df_accepted.columns = ['order_id', 'Accepted', 'timestamp_accepted']

#Merge the dfs together
dfs_to_merge = [df_order_created, df_cobra, df_cobrm, df_ocara, df_accepted]
df_final = reduce(lambda  left, right: pd.merge(left, right,on=['order_id'], how='left'), dfs_to_merge).fillna('NA')
df_final.head()

,order_id,pickup_location_hex_8,hour,yyyymmdd,timestamp,event_type,new_cancel_reason,COBRA,timestamp_cobra,COBRM,timestamp_cobrm,OCARA,timestamp_ocara,Accepted,timestamp_accepted
0,643304506b042d72ccd30197,88618920b1fffff,00,20230410,2023-04-10 00:00:40.845,order_requested,no_cancellation,NA,NA,NA,NA,OCARA,2023-04-10 00:11:11.936000,accepted,2023-04-10 00:00:49.505000
1,64330485ddfc230482e14b7d,8861892097fffff,00,20230410,2023-04-10 00:01:33.894,order_requested,no_cancellation,NA,NA,NA,NA,OCARA,2023-04-10 00:03:48.912000,accepted,2023-04-10 00:03:23.644000
2,643304946b042d72ccd301de,8861892723fffff,00,20230410,2023-04-10 00:01:48.607,order_requested,no_cancellation,COBRA,2023-04-10 00:01:51.353000,NA,NA,NA,NA,NA,NA
3,64330495373d85214287ba30,88618920b9fffff,00,20230410,2023-04-10 00:01:49.254,order_requested,no_cancellation,NA,NA,NA,NA,OCARA,2023-04-10 00:10:36.752000,accepted,2023-04-10 00:01:59.065000
4,643304aaddfc230482e14bb9,8861892081fffff,00,20230410,2023-04-10 00:02:10.447,order_requested,no_cancellation,COBRA,2023-04-10 00:02:15.417000,NA,NA,NA,NA,NA,NA


In [17]:
#Filter on the respective order_status
df_cobra = df_final[df_final['COBRA'] == 'COBRA']
df_ocara = df_final[df_final['OCARA'] == 'OCARA']
df_cobrm = df_final[df_final['COBRM'] == 'COBRM']
df_accepted = df_final[df_final['Accepted'] == 'accepted']

#Calculate TTC for cobrm, ocara, and cobra and tta
df_cobra['ttc_cobra'] = pd.to_datetime(df_cobra['timestamp_cobra']) - pd.to_datetime(df_cobra['timestamp'])
df_cobra['ttc_cobra'] = df_cobra['ttc_cobra'].apply(lambda x: x.total_seconds())

df_cobrm['ttc_cobrm'] = pd.to_datetime(df_cobrm['timestamp_cobrm']) - pd.to_datetime(df_cobrm['timestamp'])
df_cobrm['ttc_cobrm'] = df_cobrm['ttc_cobrm'].apply(lambda x: x.total_seconds())

df_ocara['ttc_ocara'] = pd.to_datetime(df_ocara['timestamp_ocara']) - pd.to_datetime(df_ocara['timestamp'])
df_ocara['ttc_ocara'] = df_ocara['ttc_ocara'].apply(lambda x: x.total_seconds())

df_accepted['tta'] = pd.to_datetime(df_accepted['timestamp_accepted']) - pd.to_datetime(df_accepted['timestamp'])
df_accepted['tta'] = df_accepted['tta'].apply(lambda x: x.total_seconds())

#Filter only on the new columns made
df_cobra = df_cobra[['order_id', 'ttc_cobra']]
df_cobrm = df_cobrm[['order_id', 'ttc_cobrm']]
df_ocara = df_ocara[['order_id', 'ttc_ocara']]
df_accepted = df_accepted[['order_id', 'tta']]

#Merge the dfs together
dfs_to_merge = [df_final, df_cobra, df_cobrm, df_ocara, df_accepted]
df_final = reduce(lambda  left, right: pd.merge(left, right,on=['order_id'], how='left'), dfs_to_merge)
df_final.head()

,order_id,pickup_location_hex_8,hour,yyyymmdd,timestamp,event_type,new_cancel_reason,COBRA,timestamp_cobra,COBRM,timestamp_cobrm,OCARA,timestamp_ocara,Accepted,timestamp_accepted,ttc_cobra,ttc_cobrm,ttc_ocara,tta
0,643304506b042d72ccd30197,88618920b1fffff,00,20230410,2023-04-10 00:00:40.845,order_requested,no_cancellation,NA,NA,NA,NA,OCARA,2023-04-10 00:11:11.936000,accepted,2023-04-10 00:00:49.505000,NaN,NaN,631.091,8.660
1,64330485ddfc230482e14b7d,8861892097fffff,00,20230410,2023-04-10 00:01:33.894,order_requested,no_cancellation,NA,NA,NA,NA,OCARA,2023-04-10 00:03:48.912000,accepted,2023-04-10 00:03:23.644000,NaN,NaN,135.018,109.750
2,643304946b042d72ccd301de,8861892723fffff,00,20230410,2023-04-10 00:01:48.607,order_requested,no_cancellation,COBRA,2023-04-10 00:01:51.353000,NA,NA,NA,NA,NA,NA,2.746,NaN,NaN,NaN
3,64330495373d85214287ba30,88618920b9fffff,00,20230410,2023-04-10 00:01:49.254,order_requested,no_cancellation,NA,NA,NA,NA,OCARA,2023-04-10 00:10:36.752000,accepted,2023-04-10 00:01:59.065000,NaN,NaN,527.498,9.811
4,643304aaddfc230482e14bb9,8861892081fffff,00,20230410,2023-04-10 00:02:10.447,order_requested,no_cancellation,COBRA,2023-04-10 00:02:15.417000,NA,NA,NA,NA,NA,NA,4.970,NaN,NaN,NaN


In [18]:
#Create a date column
df_final['date'] = df_final['yyyymmdd'].astype(str).apply(lambda x: x[6:])
df_final['count'] = 1

#Get average ttc, tta
df_cobra_grouped = df_final[df_final['COBRA'] == 'COBRA'].groupby(['yyyymmdd', 'hour'])['ttc_cobra'].median().reset_index()
df_ocara_grouped = df_final[df_final['OCARA'] == 'OCARA'].groupby(['yyyymmdd', 'hour'])['ttc_ocara'].median().reset_index()
df_cobrm_grouped = df_final[df_final['COBRM'] == 'COBRM'].groupby(['yyyymmdd', 'hour'])['ttc_cobrm'].median().reset_index()
df_accepted_grouped = df_final[df_final['Accepted'] == 'accepted'].groupby(['yyyymmdd', 'hour'])['tta'].median().reset_index()

#Merge the dfs together
dfs_to_merge = [df_cobra_grouped, df_cobrm_grouped, df_ocara_grouped, df_accepted_grouped]
df_metrics_final = reduce(lambda  left, right: pd.merge(left, right,on=['yyyymmdd', 'hour'], how='left'), dfs_to_merge).fillna('NA')
df_metrics_final.head()

,yyyymmdd,hour,ttc_cobra,ttc_cobrm,ttc_ocara,tta
0,20230410,00,30.8520,NA,293.9400,12.3770
1,20230410,01,29.4990,NA,334.2420,11.3710
2,20230410,02,16.2855,66.352,178.0860,9.4950
3,20230410,03,86.7055,101.7845,312.9215,11.7645
4,20230410,04,104.4680,NA,240.8480,19.8205


In [19]:
#Merge ttc with metrics_df
metrics_df = metrics_df.merge(df_metrics_final, on = ['yyyymmdd', 'hour'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta
0,20230410,00,0.3367,0.2551,0.0000,0.3929,0.0102,0.0,196,1.260364,4.857143,25.0,30.8520,NA,293.9400,12.3770
1,20230410,01,0.2917,0.0833,0.0000,0.5833,0.0417,0.0,48,1.633357,5.857143,23.0,29.4990,NA,334.2420,11.3710
2,20230410,02,0.4167,0.1111,0.0278,0.4167,0.0278,0.0,36,1.453200,6.133333,13.0,16.2855,66.352,178.0860,9.4950
3,20230410,03,0.2903,0.2581,0.1613,0.2419,0.0323,0.0,62,1.426933,5.066667,8.0,86.7055,101.7845,312.9215,11.7645
4,20230410,04,0.2000,0.2889,0.0000,0.4222,0.0889,0.0,45,2.018526,6.105263,5.5,104.4680,NA,240.8480,19.8205


In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd, pickup_location_latitude, pickup_location_longitude,
order_status, cancel_reason, updated_epoch, event_type, captain_id, spd_fraud_flag, pickup_location_hex_8, estimated_accept_to_pickup_distance
from orders.order_logs_immutable
where yyyymmdd in ('20230427', '20230428', '20230429')
and service_obj_service_name = 'Auto'
and event_type in ('accepted', 'rider_busy', 'rider_reject')
and pickup_location_hex_8 in ('886189258dfffff', '8861892513fffff', '88618925c3fffff', '88618925ebfffff', '88618925cbfffff', '88618925d1fffff', '886189258bfffff',
 '8861892425fffff', '88618925e3fffff', '8861892589fffff', '886189251bfffff', '8861892599fffff', '88618925c7fffff', '8861892427fffff', '8861892587fffff',
   '8861892585fffff', '88618925ddfffff', '8861892581fffff', '88618925d3fffff', '88618925c9fffff', '88618925c5fffff', '88618925d5fffff', '886189259dfffff',
     '88618925e9fffff', '88618925e1fffff', '88618925cdfffff', '8861892583fffff', '88618925c1fffff', '88618925d7fffff', '8861892519fffff', '8861892517fffff',
       '88618925edfffff', '8861892511fffff')
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_apr_copy = pd.read_sql(q, connection)

In [4]:
df_apr_copy.to_csv('upper_zone_pings.csv', index = False)

In [5]:
#Load and view dataset
df_apr = df_apr_copy.copy()
df_apr.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,estimated_accept_to_pickup_distance
0,644273c2603a9f0f9a89bed1,NaN,8861892513fffff,12.934567,77.629794,Bangalore,88618925e9fffff,8861892e9bfffff,12.982190,77.608349,12.939368,77.629742,170010,17,1700,Auto,17:00,5,20230421,new,None,1682076652760,rider_reject,61fcf64d89c809376b926a99,None,88618925e9fffff,381.5
1,644275a31c6d14472f4b8290,NaN,8861892593fffff,12.935492,77.573809,Bangalore,8861892599fffff,88618924abfffff,12.905683,77.572124,12.929711,77.585007,170811,17,1700,Auto,17:00,5,20230421,new,None,1682077165792,rider_reject,60f1546c8f34797f8c17d58f,None,8861892599fffff,1668.8
2,644275bba77ff3688a1da4bd,NaN,8861892421fffff,12.912398,77.635745,Bangalore,886189251bfffff,88618925c3fffff,12.926737,77.613846,12.923150,77.639069,170835,17,1700,Auto,17:00,5,20230421,new,None,1682077295334,rider_reject,62f15ae09b353b34e3df72a2,None,8861892425fffff,1454.3
3,6442767a61dc4141c7a71317,NaN,8861892437fffff,12.912416,77.610023,Bangalore,88618925cbfffff,8861892407fffff,12.898358,77.628868,12.914371,77.614304,171146,17,1700,Auto,17:00,5,20230421,new,None,1682077347282,rider_reject,642e8f37f7aa953b27a85379,None,88618925cbfffff,747.9
4,644277ba61dc4141c7a71552,NaN,88618925d3fffff,12.916881,77.586334,Bangalore,88618925d3fffff,8861892437fffff,12.914722,77.610251,12.917548,77.591845,171706,17,1715,Auto,17:00,5,20230421,new,None,1682077658339,rider_busy,60e82ba3569b26d0fe744faf,None,88618925d3fffff,901.7


In [6]:
df_apr.groupby(['yyyymmdd'])['estimated_accept_to_pickup_distance'].median().reset_index()

,yyyymmdd,estimated_accept_to_pickup_distance
0,20230420,1277.30
1,20230421,1238.50
2,20230422,1238.90
3,20230427,1329.30
4,20230428,1309.30
5,20230429,1271.35


In [7]:
df_apr.groupby(['yyyymmdd'])['estimated_accept_to_pickup_distance'].quantile(0.9).reset_index()

,yyyymmdd,estimated_accept_to_pickup_distance
0,20230420,1875.00
1,20230421,1864.50
2,20230422,1866.10
3,20230427,1874.67
4,20230428,1871.30
5,20230429,1856.20


In [9]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, captain_id, spd_fraud_flag, pickup_location_hex_8, estimated_accept_to_pickup_distance, map_riders
from orders.order_logs_immutable
where yyyymmdd in ('20230420', '20230421', '20230422', '20230427', '20230428', '20230429')
and service_obj_service_name = 'Auto'
and event_type in ('order_create', 'order_refreshed')
and pickup_location_hex_8 in ('886189258dfffff', '8861892513fffff', '88618925c3fffff', '88618925ebfffff', '88618925cbfffff', '88618925d1fffff', '886189258bfffff',
 '8861892425fffff', '88618925e3fffff', '8861892589fffff', '886189251bfffff', '8861892599fffff', '88618925c7fffff', '8861892427fffff', '8861892587fffff',
   '8861892585fffff', '88618925ddfffff', '8861892581fffff', '88618925d3fffff', '88618925c9fffff', '88618925c5fffff', '88618925d5fffff', '886189259dfffff',
     '88618925e9fffff', '88618925e1fffff', '88618925cdfffff', '8861892583fffff', '88618925c1fffff', '88618925d7fffff', '8861892519fffff', '8861892517fffff',
       '88618925edfffff', '8861892511fffff')
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_apr_copy = pd.read_sql(q, connection)

In [10]:
#Load and view dataset
df_apr = df_apr_copy.copy()
df_apr.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,estimated_accept_to_pickup_distance,map_riders
0,6441298b1d65051ed23bc72c,NaN,,NaN,NaN,Bangalore,8861892589fffff,8861892589fffff,12.933153,77.608294,12.936349,77.606255,173115,17,1730,Auto,17:59,4,20230420,new,None,1681992076016,order_create,,None,8861892589fffff,NaN,"[""630f140dfb9d0c059196f5d4"",""63d9d788aa0c7efd4..."
1,64412e4059513b224af40a6e,NaN,8861892589fffff,12.935469,77.608535,Bangalore,88618925c7fffff,88618925cdfffff,12.925863,77.624625,12.935094,77.618736,175120,17,1745,Auto,17:59,4,20230420,new,None,1681993427997,order_refreshed,61136aa0f7da3d07d0131a13,None,88618925c7fffff,1691.6,"[""62d299e47fa81e58424d5918"",""61028d206cd49272f..."
2,64412dc659513b224af40994,NaN,,NaN,NaN,Bangalore,88618925c9fffff,8861892457fffff,12.874682,77.619433,12.916100,77.619657,174918,17,1745,Auto,17:59,4,20230420,new,None,1681993159619,order_create,,None,88618925c9fffff,NaN,"[""63270879426a42ab8a329b33"",""61121e2ef3519d0ce..."
3,6441251fd1e1ab77e4e66544,NaN,88618925d5fffff,12.930415,77.602387,Bangalore,886189258bfffff,88618925e9fffff,12.936769,77.627151,12.932868,77.602135,171223,17,1700,Auto,17:00,4,20230420,new,None,1681990987010,order_refreshed,62077f14179f706a0bb939d8,None,8861892589fffff,1931.4,"[""61335e368709d20d3993af3d"",""62077f14179f706a0..."
4,64412f9b2c5940022415c179,NaN,8861892587fffff,12.948242,77.597041,Bangalore,8861892585fffff,88618925c7fffff,12.935722,77.614970,12.949184,77.606194,175707,17,1745,Auto,17:59,4,20230420,new,None,1681993753536,order_refreshed,610381c36c5e5fa2c8bffc27,None,8861892585fffff,1841.4,"[""60efdba40d1ac7746e9e189e"",""61f5711a6916b8ad5..."


In [11]:
#Get the number of captains mapped
df_apr['map_riders'] = df_apr['map_riders'].apply(lambda x: json.loads(x))
df_captains_mapped_grouped = df_apr.groupby(['yyyymmdd', 'order_id'])['map_riders'].apply(list).reset_index()
df_captains_mapped_grouped['map_riders'] = df_captains_mapped_grouped['map_riders'].apply(lambda x: list(set([item for sublist in x for item in sublist])))
master_list = []
for x in df_captains_mapped_grouped['map_riders']:
    captains_list = []
    for captain in x:
        if captain in captains_list:
            continue
        else:
            captains_list.append(captain)
    master_list.append(captains_list)
df_captains_mapped_grouped['map_riders'] = master_list
df_captains_mapped_grouped['number_of_mapped_captains'] = df_captains_mapped_grouped['map_riders'].apply(lambda x: len(x))
df_captains_mapped_grouped.head()

,yyyymmdd,order_id,map_riders,number_of_mapped_captains
0,20230420,644122381c6d14472f498524,"[624ed6bb96ae680c9ef88aa3, 6432022e04c7f5159f1...",70
1,20230420,6441223c246f7347ed4ee994,"[60e2e2df569b26cb5a717654, 631eceff9220544f5da...",34
2,20230420,6441223c9e36a256b02295d1,"[60faa4b68f3479e2251ca479, 61bcd0e3f3de6f9b224...",20
3,20230420,6441223d67e55167dd9ebca6,"[5a1eb2630e5ee07cb40a4ac8, 60e84cb409aa9923d64...",72
4,20230420,6441223f2937c94eff8091aa,"[62d76b330d3baea96be408b5, 5e43800ac478386aff1...",55


In [12]:
df_captains_mapped_grouped.groupby(['yyyymmdd'])['number_of_mapped_captains'].median().reset_index()


,yyyymmdd,number_of_mapped_captains
0,20230420,30.0
1,20230421,32.0
2,20230422,27.0
3,20230427,30.0
4,20230428,34.0
5,20230429,31.0


In [13]:
df_captains_mapped_grouped.groupby(['yyyymmdd'])['number_of_mapped_captains'].quantile(0.9).reset_index()


,yyyymmdd,number_of_mapped_captains
0,20230420,47.0
1,20230421,49.0
2,20230422,45.0
3,20230427,47.0
4,20230428,53.0
5,20230429,50.0


In [22]:
#Create a count column
df_apr['count'] = 1

#Create a column to identify total pings and accepted ping
df_apr['total_pings'] = np.where(df_apr['event_type'].isin(['accepted', 'rider_busy', 'rider_reject']), 1, 0)
df_apr['accepted_pings'] = np.where(df_apr['event_type'].isin(['accepted']), 1, 0)
df_me = df_apr[(df_apr['order_status'] == 'dropped') & (df_apr['spd_fraud_flag'] != True)]

#Groupby on date and hour to get the APR and ME on an hourly basis
apr_hyderabad = df_apr.groupby(['yyyymmdd', 'hour'])[['total_pings', 'accepted_pings']].sum().reset_index()
apr_hyderabad['APR'] = round(apr_hyderabad['accepted_pings']/apr_hyderabad['total_pings'], 4)
me_hyderabad = df_me.groupby(['yyyymmdd', 'hour'])['order_id'].nunique().reset_index()
me_hyderabad = me_hyderabad.rename(columns={'order_id':'net_orders'})
apr_hyderabad = apr_hyderabad.merge(me_hyderabad, on = ['yyyymmdd', 'hour'], how = 'left')
apr_hyderabad['ME'] = round(apr_hyderabad['total_pings']/apr_hyderabad['net_orders'], 2)

#Merge APR and ME with metrics_df
metrics_df = metrics_df.merge(apr_hyderabad, on = ['yyyymmdd', 'hour'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta,total_pings,accepted_pings,APR,net_orders,ME
0,20230410,00,0.3367,0.2551,0.0000,0.3929,0.0102,0.0,196,1.260364,4.857143,25.0,30.8520,NA,293.9400,12.3770,761,174,0.2286,70,10.87
1,20230410,01,0.2917,0.0833,0.0000,0.5833,0.0417,0.0,48,1.633357,5.857143,23.0,29.4990,NA,334.2420,11.3710,157,58,0.3694,26,6.04
2,20230410,02,0.4167,0.1111,0.0278,0.4167,0.0278,0.0,36,1.453200,6.133333,13.0,16.2855,66.352,178.0860,9.4950,76,32,0.4211,14,5.43
3,20230410,03,0.2903,0.2581,0.1613,0.2419,0.0323,0.0,62,1.426933,5.066667,8.0,86.7055,101.7845,312.9215,11.7645,191,44,0.2304,15,12.73
4,20230410,04,0.2000,0.2889,0.0000,0.4222,0.0889,0.0,45,2.018526,6.105263,5.5,104.4680,NA,240.8480,19.8205,207,33,0.1594,18,11.50


In [23]:
#Query data to calculate gsr
q = """
select 
        pickup_location_hex_8, yyyymmdd, hour,
        count(distinct order_id) as Gross_Orders,
        count(case when (time_diff >=20 or time_diff is NULL) then customer_id end) as GSR,
        count(distinct case when event_type in ('dropped') and spd_fraud_flag != True then order_id end) as net_orders,
        (case when count(distinct case when event_type in ('dropped') and spd_fraud_flag != True then order_id end) = 0 then 0 else
        sum(case when event_type in ('accepted', 'rider_busy', 'rider_reject') then 1 else 0 end)
        /cast(count(distinct case when event_type in ('dropped') and spd_fraud_flag != True then order_id end) as real) end) as ME
from
(
select
        *, 
        date_diff('minute',  LAG(FROM_UNIXTIME(CAST(epoch AS double) /1000)) OVER(PARTITION BY yyyymmdd, hour, city_name, customer_id, order_type, customer_location_hex_8
        ORDER BY yyyymmdd, customer_id, city_name, customer_location_hex_8, order_type, epoch), FROM_UNIXTIME(CAST(epoch AS double) /1000)) as time_diff,
        date_diff('minute',  LAG(FROM_UNIXTIME(CAST(epoch AS double) /1000)) OVER(PARTITION BY yyyymmdd, hour, city_name, customer_id, order_type, order_status, customer_location_hex_8, spd_fraud_flag 
        ORDER BY yyyymmdd, customer_id, city_name, customer_location_hex_8, order_type, epoch) , FROM_UNIXTIME(CAST(epoch AS double) /1000)) as time_diff_drop
from
(
select 
    order_id, city_name, order_type, cancel_reason, yyyymmdd, customer_id, epoch, substr(hhmmss,1,2) as hour,
    row_number() over (partition by order_id order by updated_epoch desc) as Rank_1,
    
    event_type, order_status, captain_id, pickup_cluster, customer_location_hex_8, spd_fraud_flag, pickup_location_hex_8

from
    orders.order_logs_immutable 
where
    yyyymmdd >= '20230410'
    and yyyymmdd <= '20230423'
    and order_type IN ('app')
    and pickup_location_hex_8 in ('8861892545fffff', '886189254bfffff', '8861892541fffff', '88618920d5fffff', '8861892099fffff', '8861892085fffff', '886189255dfffff',
 '8861892087fffff', '8861892543fffff', '88618920e3fffff', '88618920bbfffff', '886189254dfffff', '88618920dbfffff', '8861892729fffff', '8861892089fffff',
  '8861892093fffff', '886189209bfffff', '88618920abfffff', '8861892767fffff', '8861892081fffff', '88618920a9fffff', '886189208dfffff', '886189272dfffff',
   '8861892723fffff', '8861892083fffff', '8861892549fffff', '88618920b9fffff', '8861892091fffff', '8861892095fffff', '8861892721fffff', '88618920b1fffff',
    '8861892727fffff', '886189209dfffff', '8861892097fffff', '8861892725fffff', '88618920bdfffff')
))
group by 1,2,3
"""

df_gsr_copy = pd.read_sql(q, connection)

In [24]:
#Create a copy of the dataset and view it
df_gsr = df_gsr_copy.copy()
df_gsr.head()

,pickup_location_hex_8,yyyymmdd,hour,Gross_Orders,GSR,net_orders,ME
0,8861892097fffff,20230420,21,120,88,40,32.200000
1,88618920bdfffff,20230420,05,4,2,1,5.000000
2,8861892097fffff,20230411,10,55,40,17,7.470588
3,8861892541fffff,20230411,07,16,15,11,3.727273
4,8861892549fffff,20230412,18,87,53,11,68.090910


In [25]:
#Calculate GSR2Net
df_gsr_grouped = df_gsr.groupby(['yyyymmdd', 'hour'])[['GSR', 'net_orders']].sum().reset_index()
df_gsr_grouped['GSR2Net'] = round(df_gsr_grouped['net_orders']/df_gsr_grouped['GSR'], 4)

#Merge GSR with metrics_df
metrics_df = metrics_df.merge(df_gsr_grouped[['yyyymmdd', 'hour', 'GSR', 'GSR2Net']], on = ['yyyymmdd', 'hour'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta,total_pings,accepted_pings,APR,net_orders,ME,GSR,GSR2Net
0,20230410,00,0.3367,0.2551,0.0000,0.3929,0.0102,0.0,196,1.260364,4.857143,25.0,30.8520,NA,293.9400,12.3770,761,174,0.2286,70,10.87,158,0.4430
1,20230410,01,0.2917,0.0833,0.0000,0.5833,0.0417,0.0,48,1.633357,5.857143,23.0,29.4990,NA,334.2420,11.3710,157,58,0.3694,26,6.04,44,0.5909
2,20230410,02,0.4167,0.1111,0.0278,0.4167,0.0278,0.0,36,1.453200,6.133333,13.0,16.2855,66.352,178.0860,9.4950,76,32,0.4211,14,5.43,32,0.4375
3,20230410,03,0.2903,0.2581,0.1613,0.2419,0.0323,0.0,62,1.426933,5.066667,8.0,86.7055,101.7845,312.9215,11.7645,191,44,0.2304,15,12.73,46,0.3261
4,20230410,04,0.2000,0.2889,0.0000,0.4222,0.0889,0.0,45,2.018526,6.105263,5.5,104.4680,NA,240.8480,19.8205,207,33,0.1594,18,11.50,38,0.4737


In [26]:
#Query data from supply_login_servicelevel_kpi
q = """
select
        location as pickup_location_hex_8, yyyymmdd, substr(hhmm, 1, 2) as hour,
        sum(case when cast(status AS varchar) IN ('2','3','6','7','8','10') then duration end) as login_time,
        sum(case when cast(status AS varchar) IN ('2') then duration end) as idle_time
from 
(select 
        a.*, city, cluster
        
from 
        hive.datasets.supplycursory_history a 
        left join datasets.city_cluster_hex b 
        on a.location = b.hex_id and a.resolution = cast(b.resolution as varchar)
where 
        yyyymmdd >= '20230410'
        and yyyymmdd <= '20230423'
        and city in ('Bangalore')
        and a.resolution = '8' 
        and service in ('link')
        and location in ('8861892545fffff', '886189254bfffff', '8861892541fffff', '88618920d5fffff', '8861892099fffff', '8861892085fffff', '886189255dfffff',
 '8861892087fffff', '8861892543fffff', '88618920e3fffff', '88618920bbfffff', '886189254dfffff', '88618920dbfffff', '8861892729fffff', '8861892089fffff',
  '8861892093fffff', '886189209bfffff', '88618920abfffff', '8861892767fffff', '8861892081fffff', '88618920a9fffff', '886189208dfffff', '886189272dfffff',
   '8861892723fffff', '8861892083fffff', '8861892549fffff', '88618920b9fffff', '8861892091fffff', '8861892095fffff', '8861892721fffff', '88618920b1fffff',
    '8861892727fffff', '886189209dfffff', '8861892097fffff', '8861892725fffff', '88618920bdfffff')
)
group by 1,2,3
"""

#Load data into pandas table
df_tse_copy = pd.read_sql(q, connection)

In [27]:
#Create a copy of the df and view it
df_tse = df_tse_copy.copy()
df_tse.head()

,pickup_location_hex_8,yyyymmdd,hour,login_time,idle_time
0,8861892549fffff,20230423,22,2542.0,194.0
1,886189209dfffff,20230421,10,15348.0,2201.0
2,8861892549fffff,20230423,19,6336.0,586.0
3,886189209dfffff,20230420,22,9417.0,2500.0
4,8861892549fffff,20230420,10,32339.0,17996.0


In [28]:
#Calculate time_spent_earning
df_tse_grouped = df_tse.groupby(['yyyymmdd', 'hour'])[['login_time', 'idle_time']].sum().reset_index()
df_tse_grouped['time_spent_earning'] = round((df_tse_grouped['login_time'] - df_tse_grouped['idle_time'])/df_tse_grouped['login_time'], 4)
df_tse_grouped.head()

,yyyymmdd,hour,login_time,idle_time,time_spent_earning
0,20230410,00,513318.0,289845.0,0.4354
1,20230410,01,330644.0,232282.0,0.2975
2,20230410,02,218709.0,174282.0,0.2031
3,20230410,03,125778.0,94549.0,0.2483
4,20230410,04,100042.0,70534.0,0.2950


In [29]:
#Merge GSR with metrics_df
metrics_df = metrics_df.merge(df_tse_grouped, on = ['yyyymmdd', 'hour'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta,total_pings,accepted_pings,APR,net_orders,ME,GSR,GSR2Net,login_time,idle_time,time_spent_earning
0,20230410,00,0.3367,0.2551,0.0000,0.3929,0.0102,0.0,196,1.260364,4.857143,25.0,30.8520,NA,293.9400,12.3770,761,174,0.2286,70,10.87,158,0.4430,513318.0,289845.0,0.4354
1,20230410,01,0.2917,0.0833,0.0000,0.5833,0.0417,0.0,48,1.633357,5.857143,23.0,29.4990,NA,334.2420,11.3710,157,58,0.3694,26,6.04,44,0.5909,330644.0,232282.0,0.2975
2,20230410,02,0.4167,0.1111,0.0278,0.4167,0.0278,0.0,36,1.453200,6.133333,13.0,16.2855,66.352,178.0860,9.4950,76,32,0.4211,14,5.43,32,0.4375,218709.0,174282.0,0.2031
3,20230410,03,0.2903,0.2581,0.1613,0.2419,0.0323,0.0,62,1.426933,5.066667,8.0,86.7055,101.7845,312.9215,11.7645,191,44,0.2304,15,12.73,46,0.3261,125778.0,94549.0,0.2483
4,20230410,04,0.2000,0.2889,0.0000,0.4222,0.0889,0.0,45,2.018526,6.105263,5.5,104.4680,NA,240.8480,19.8205,207,33,0.1594,18,11.50,38,0.4737,100042.0,70534.0,0.2950


In [30]:
#Create a copy of the dataset and view it
df_captains_mapped = df_copy.copy()
df_captains_mapped.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8
0,643bb6b1ee623a70eee720c4,1.507,Bangalore,8861892085fffff,12.947007,77.722290,141953,14,1415,Link,14:00,7,20230416,dropped,None,6.0,"[""63ee2f84e370d6dd6c59ef90"",""6162e9ef77c7e6b1e...",8861892085fffff
1,643bb89b6b8317383f414cab,1.853,Bangalore,88618920bbfffff,12.950012,77.701265,142803,14,1415,Link,14:00,7,20230416,dropped,None,8.0,"[""623dec2579d2674e4fb07f51"",""607a6153255e27cc6...",88618920bbfffff
2,643bbbf2e0382b532b65ba26,1.789,Bangalore,88618920b1fffff,12.956978,77.702780,144218,14,1430,Link,14:59,7,20230416,customerCancelled,Customer asked to cancel,7.0,"[""5e4b95927ea425a39b6cb910"",""63c8b353ec19f00c6...",88618920b1fffff
3,643bbcf8cac0523bb40a0e9f,1.583,Bangalore,886189254bfffff,12.919333,77.671005,144640,14,1445,Link,14:59,7,20230416,dropped,None,6.0,"[""63d1e080e00e30fe287716ef"",""606f124d7761609b7...",886189254bfffff
4,643bbdb8259b2f4d1a7b4cb9,NaN,Bangalore,88618920b1fffff,12.954740,77.700409,144952,14,1445,Link,14:59,7,20230416,customerCancelled,order cancelled before rider accepted,0.0,"[""63ff5aac24d5dc39dfcddaa9"",""5ca7164a54bc7263f...",88618920b1fffff


In [31]:
#Get the number of captains mapped
df_captains_mapped['map_riders'] = df_captains_mapped['map_riders'].apply(lambda x: json.loads(x))
df_captains_mapped_grouped = df_captains_mapped.groupby(['yyyymmdd', 'hour'])['map_riders'].apply(list).reset_index()
df_captains_mapped_grouped['map_riders'] = df_captains_mapped_grouped['map_riders'].apply(lambda x: list(set([item for sublist in x for item in sublist])))
master_list = []
for x in df_captains_mapped_grouped['map_riders']:
    captains_list = []
    for captain in x:
        if captain in captains_list:
            continue
        else:
            captains_list.append(captain)
    master_list.append(captains_list)
df_captains_mapped_grouped['map_riders'] = master_list
df_captains_mapped_grouped['number_of_mapped_captains'] = df_captains_mapped_grouped['map_riders'].apply(lambda x: len(x))
metrics_df = metrics_df.merge(df_captains_mapped_grouped[['yyyymmdd', 'hour', 'number_of_mapped_captains']], 
on = ['yyyymmdd', 'hour'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta,total_pings,accepted_pings,APR,net_orders,ME,GSR,GSR2Net,login_time,idle_time,time_spent_earning,number_of_mapped_captains
0,20230410,00,0.3367,0.2551,0.0000,0.3929,0.0102,0.0,196,1.260364,4.857143,25.0,30.8520,NA,293.9400,12.3770,761,174,0.2286,70,10.87,158,0.4430,513318.0,289845.0,0.4354,398
1,20230410,01,0.2917,0.0833,0.0000,0.5833,0.0417,0.0,48,1.633357,5.857143,23.0,29.4990,NA,334.2420,11.3710,157,58,0.3694,26,6.04,44,0.5909,330644.0,232282.0,0.2975,197
2,20230410,02,0.4167,0.1111,0.0278,0.4167,0.0278,0.0,36,1.453200,6.133333,13.0,16.2855,66.352,178.0860,9.4950,76,32,0.4211,14,5.43,32,0.4375,218709.0,174282.0,0.2031,120
3,20230410,03,0.2903,0.2581,0.1613,0.2419,0.0323,0.0,62,1.426933,5.066667,8.0,86.7055,101.7845,312.9215,11.7645,191,44,0.2304,15,12.73,46,0.3261,125778.0,94549.0,0.2483,69
4,20230410,04,0.2000,0.2889,0.0000,0.4222,0.0889,0.0,45,2.018526,6.105263,5.5,104.4680,NA,240.8480,19.8205,207,33,0.1594,18,11.50,38,0.4737,100042.0,70534.0,0.2950,62


In [32]:
metrics_df.to_csv('bangalore_bellandur_sourcing_metrics.csv', index = False)